In [1]:
import requests
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import csv
import os
from dotenv import load_dotenv
import time

# FMP News API Code

In [2]:
# loading in api file
load_dotenv(r"api.env")

# Obtaining key for API file
fmp_api_key = os.getenv("fmp_api_key")
if not fmp_api_key:
    raise ValueError("No API key set for fmp_api_key in .env file")

print("API key loaded successfully from .env file.")

ValueError: No API key set for fmp_api_key in .env file

In [ ]:
# Setting up dictionaries in lists to facilitate looping through
stock_news = [
    {"symbol": "AMZN", "ipo_date": "1997-05-15"},
    {"symbol": "AAPL", "ipo_date": "1980-12-12"},
    {"symbol": "GOOG", "ipo_date": "2004-08-19"},
    {"symbol": "MSFT", "ipo_date": "1986-03-13"},
    {"symbol": "META", "ipo_date": "2012-05-18"},
    {"symbol": "NVDA", "ipo_date": "1999-01-22"}
]

press_releases = [
    {"company": "AMZN", "ipo_date": "1997-05-15"},
    {"company": "AAPL", "ipo_date": "1980-12-12"},
    {"company": "GOOG", "ipo_date": "2004-08-19"},
    {"company": "MSFT", "ipo_date": "1986-03-13"},
    {"company": "META", "ipo_date": "2012-05-18"},
    {"company": "NVDA", "ipo_date": "1999-01-22"}
]

historical_social_sentiment = [
    {"symbol": "AMZN", "ipo_date": "1997-05-15"},
    {"symbol": "AAPL", "ipo_date": "1980-12-12"},
    {"symbol": "GOOG", "ipo_date": "2004-08-19"},
    {"symbol": "MSFT", "ipo_date": "1986-03-13"},
    {"symbol": "META", "ipo_date": "2012-05-18"},
    {"symbol": "NVDA", "ipo_date": "1999-01-22"}
]




# setting up lists to loop through for sentiment score and weighted scores analysis
stock_news_datasets = [
    "AMZN_news_data.csv",
    "AAPL_news_data.csv",
    "GOOG_news_data.csv",
    "MSFT_news_data.csv",
    "META_news_data.csv",
    "NVDA_news_data.csv"
]

news_press_releases_datasets = [
    "AMZN_press_release_data.csv",
    "AAPL_press_release_data.csv",
    "GOOG_press_release_data.csv",
    "MSFT_press_release_data.csv",
    "META_press_release_data.csv",
    "NVDA_press_release_data.csv"
]

social_sentiments_datasets = [
    "AMZN_social_sentiment_data.csv",
    "AAPL_social_sentiment_data.csv",
    "GOOG_social_sentiment_data.csv",
    "MSFT_social_sentiment_data.csv",
    "META_social_sentiment_data.csv",
    "NVDA_social_sentiment_data.csv"
]

# Stock News Data Pull 

In [ ]:
# Obtaining today's date
current_date =date.today()

# Obtaining stock news data for each tech company
for news in stock_news:

    # setting up url path & parameter values
    url = f"https://financialmodelingprep.com/api/v3/stock_news?"
    page = 0
    limit = 2000

    # creating empty list to capture news data
    all_news = []

    # looping through pages of data to obtain stock news data
    while True:
        
        # applying parameter values to required params
        params = {
        "apikey" : fmp_api_key,
        "tickers" : news['symbol'],
        "page": page,
        'from': news['ipo_date'],
        'to': current_date,
        'limit': limit
        }

        # requesting data
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {news['symbol']} on page {page}: {e}")
            break

        # setting successfully requested data to variable data
        data = response.json()

        if not data:
            break

        # appending additional data to list
        all_news.extend(data)

        # moving to next page
        page += 1

    csv_file = f"{news['symbol']}_news_data.csv"
    headers = ["published_date", "Headline", "Brief", "URL"]
    formatted_data = []

    # writing all pulled news data to their respective companies in csv files
    if all_news:
        for record in all_news:

            date_str = record.get("publishedDate", "").split(" ")[0]  # Extract "YYYY-MM-DD" only

            formatted_data.append({
            "published_date": date_str,
            "Headline": record.get("title", ""),
            "Brief": record.get("text", ""),
            "URL": record.get("url", "")
            })

        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            writer.writeheader()
            writer.writerows(formatted_data)
        
        print(f"Data for {news['symbol']} written to {csv_file} with {len(formatted_data)} articles!")
    else:
        print(f"No data found for {news['symbol']}.")

print("All stock data successfully written to their respective CSV files.")
                
    

Data for AMZN written to AMZN_news_data.csv with 29337 articles!
Data for AAPL written to AAPL_news_data.csv with 26482 articles!
Data for GOOG written to GOOG_news_data.csv with 17780 articles!
Data for MSFT written to MSFT_news_data.csv with 16737 articles!
Data for META written to META_news_data.csv with 18438 articles!
Data for NVDA written to NVDA_news_data.csv with 19013 articles!
All stock data successfully written to their respective CSV files.


# Stock Press Release Data Pull

In [ ]:
# Obtaining stock press release data for each tech company
for press_release in press_releases:

    # setting up url path & parameter values
    url = f"https://financialmodelingprep.com/api/v3/press-releases/{press_release['company']}?"

    # setting page to zero
    page = 0

    # creating empty list to capture all press release data
    all_press_releases = []

    max_pages = 300

    # looping through pages of data to obtain stock press release data
    while page < max_pages:

        # applying parameter values to required params
        params = {
        "apikey" : fmp_api_key,
        "page": page
        }

        # requesting data
        try:
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {press_release['company']} on page {page}: {e}")
            break

        time.sleep(1)

        # setting successfully requested data to variable data
        data = response.json()
        
        print(f"Fetching data for {press_release['company']} - Page {page}...")

        if not data:
            break


        # appending additional data to list
        all_press_releases.extend(data)

        # moving to next page
        page += 1

    csv_file = f"{press_release['company']}_press_release_data.csv"
    headers = ["release_date", "Headline", "Brief"]
    formatted_data = []

    # writing all pulled press release data to their respective companies in csv files
    if all_press_releases:
        for record in all_press_releases:

            date_str = record.get("date", "").split(" ")[0]  # Extract "YYYY-MM-DD" only

            formatted_data.append({
            "release_date": date_str,
            "Headline": record.get("title", ""),
            "Brief": record.get("text", "")
            })

        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            writer.writeheader()
            writer.writerows(formatted_data)
        
        print(f"Data for {press_release['company']} written to {csv_file} with {len(formatted_data)} press releases!")
    else:
        print(f"No data found for {press_release['company']}.")

print("All stock data successfully written to their respective CSV files.")
                
    

# Twitter Sentiments Data Pull

In [ ]:
# current_date = date.today()
 
# Loop through each stock symbol to fetch sentiment data
for social_sentiment in historical_social_sentiment:
    symbol = social_sentiment["symbol"]
    ipo_date = social_sentiment["ipo_date"]
 
    # Correct API URL
    url = f"https://financialmodelingprep.com/api/v4/historical/social-sentiment?symbol={symbol}"
 
    page = 0
    all_historical_social_sentiment = []
 
    while True:
        social_params = {
            "apikey": fmp_api_key,
            "page": page
        }
 
        try:
            response = requests.get(url, params=social_params)
            response.raise_for_status()
            data = response.json()
 
            # Print API response for debugging
            print(f"\n🔍 Response for {symbol}, Page {page}: {len(data)} records")
 
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data for {symbol} on page {page}: {e}")
            break


        time.sleep(1)
 
        # Stop if no data is returned
        if not data:
            print(f"No sentiment data found for {symbol}.")
            break
 
        all_historical_social_sentiment.extend(data)
        page += 1  # Move to the next page
 
    # CSV file name (each company has its own file)
    csv_file = f"{symbol}_social_sentiment_data.csv"
 
    # Updated headers to match API response
    headers = [
        "published_date", "twitter_posts", "twitter_likes", "twitter_sentiment",
        "stock_twitter_posts", "stock_twitter_likes", "sentiment_score"
    ]
 
    formatted_data = []
 
    if all_historical_social_sentiment:
        for record in all_historical_social_sentiment:
            # Extract only the date part
            date_str = record.get("date", "").split(" ")[0]  # Extract "YYYY-MM-DD" only
 
            formatted_data.append({
                "published_date": date_str,
                "twitter_posts": record.get("twitterPosts", ""),
                "twitter_likes": record.get("twitterLikes", ""),
                "twitter_sentiment": record.get("twitterSentiment", ""),
                "stock_twitter_posts": record.get("stocktwitsPosts", ""),
                "stock_twitter_likes": record.get("stocktwitsLikes", ""),  # Fixed duplicate key
                "sentiment_score": record.get("stocktwitsSentiment", "")
            })
 
        # Write data to a CSV file for this company
        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            writer.writeheader()
            writer.writerows(formatted_data)
 
        print(f"Sentiment data for {symbol} written to {csv_file} with {len(formatted_data)} records!")
    else:
        print(f"No sentiment data found for {symbol}.")
 
print("\nAll sentiment data successfully written to CSV files.")

# Text PreProcessing & Function Creations

In [ ]:

# creating a function to clean text for VADER sentiment analysis
def text_cleaning(text, dataset):
    text = text.str.lower()
    text = text.str.replace(r"http\S+|www\S+", "", regex=True) # removing URLS
    text = text.str.replace(r"[^a-zA-Z0-9$%.,!?'\s-]", "", regex=True) # keeping relevant characters
    text = text.str.replace(r"\s+", " ", regex=True).str.strip() # removing extra spaces
    text = text.str.replace(r"\b(\d+)%", r"\1 percent", regex=True) # converting percentages
    text = text.str.replace(r"\b(\d+)M\b", r"\1 million", regex=True)  # Convert M to million
    text = text.str.replace(r"\b(\d+)B\b", r"\1 billion", regex=True)  # Convert B to billion

    # splitting texts manually based on ".!?"
    sentences = text.str.split(r'[.!?]\s+', regex=True)
    
    # creating column for the cleaned text and naming it 'cleaned'
    dataset['cleaned'] = sentences

    return dataset

In [ ]:
# creating a function to obtain sentiment scores
def sentiment_score_calculator(datasets_lists):

    # Initializing VADER sentiment analyzer
    analyzer = SentimentIntensityAnalyzer()


    # looping through datasets 
    for file in datasets_lists:
        
        # reading in file 
        news_press_release = pd.read_csv(file)

        # combining two text columns into one column to clean text in one go
        news_press_release['raw'] = news_press_release.apply(lambda row: row['Headline'] if pd.isna(row['Brief']) 
                                                        else (row['Brief'] if pd.isna(row['Headline']) 
                                                            else row['Headline'] + ". " + row['Brief']), axis=1)

        # cleaning text cleaning function with VADER appliation
        text_cleaning(news_press_release['raw'], news_press_release)


        # creating empty lists
        score_rating = []
        score = []

        # looping through each row in the column 'cleaned'
        for row in news_press_release['cleaned']:
            
            row_score = 0

            # looping through each sentence in current row and using vader to score each sentence
            for sentence in row:
                try:
                    print(sentence)
                    sentence_score = analyzer.polarity_scores(sentence)["compound"] # vader polarity score analyzer
                    print(sentence_score)
                    print("Done with sentence_score in row, onto next sentence.")

                    # accumulating each sentence's score for current row
                    row_score += sentence_score
                except RuntimeError as e:
                    print(f"Error processing sentence: {sentence}\n{e}")
                    continue

            # averaging row scores and appending to score
            avg_row_score = row_score / len(row)
            score.append(avg_row_score)

            print(f"row_score: {avg_row_score}")
            print(len(row))
            print("end of row, onto next row.")

            # classifying averaged row scores into various score ratings
            if avg_row_score > 0.05 and avg_row_score < 0.5:
                score_rating.append("weakly_positive")
                print("Weakly Positive")
            elif avg_row_score > 0.5:
                score_rating.append("strongly_positive")
                print("Strongly Positive")
            elif avg_row_score > -0.5 and avg_row_score < -0.05:
                score_rating.append("weakly_negative")
                print("Weakly Negative")
            elif avg_row_score < -0.5:
                score_rating.append("strongly_negative")
                print("Strongly Negative")
            else:
                score_rating.append("neutral")
                print("Neutral")

        # creating columns for sentiment and score
        news_press_release['sentiment'] = score_rating
        news_press_release['sentiment_score'] = score

        # writing the updated vader sentiment scores to each file 
        news_press_release.to_csv(file, index=False)
        print(f"Finished updating {file} with sentiment rating and sentiment score.")

In [ ]:
def aggregated_sentiments(datasets_lists):

    # looping through each dataset and applying aggregated sentiments
    for file in datasets_lists:

        if not file or not os.path.exists(file):
            print(f"Skipping missing file: {file}")
            continue

        try:
            # reading in dataset
            news_press_release_sentiment = pd.read_csv(file)
        except Exception as e:
            print(f"Error loading {file}: {e}")
        

        # Checking if either 'published_date' or 'release_date' exists and group by the first one found
        date_column = 'published_date' if 'published_date' in news_press_release_sentiment.columns else 'release_date'

        # computing average score per unique publilshed_date
        avg_scores = news_press_release_sentiment.groupby(date_column)['sentiment_score'].mean().reset_index()

        # classifying weighted daily scores into classes

        # Defining conditions for classification
        conditions = [
            avg_scores['sentiment_score'] > 0.5,
            (avg_scores['sentiment_score'] > 0.05) & (avg_scores['sentiment_score'] <= 0.5),
            (avg_scores['sentiment_score'] >= -0.05) & (avg_scores['sentiment_score'] <= 0.05),
            (avg_scores['sentiment_score'] < -0.05) & (avg_scores['sentiment_score'] >= -0.5),
            avg_scores['sentiment_score'] < -0.5
        ]

        # Defining corresponding classifications
        classifications = [
            "strongly_positive",
            "weakly_positive",
            "neutral",
            "weakly_negative",
            "strongly_negative"
        ]

        # Assigning classifications based on conditions
        avg_scores['weighted_daily_sentiment'] = np.select(conditions, classifications, default="neutral")
        
        # merging to assign classifications to each row
        news_press_release_sentiment = news_press_release_sentiment.merge(avg_scores[[date_column, 'weighted_daily_sentiment']], on=date_column, how='left')

        # writing to csv files
        news_press_release_sentiment.to_csv(file, index=False)
        print(f"Finished updating {file} with weighted daily sentiment.")


In [ ]:
# Calculating sentiments score for news and press release datasets for each company
sentiment_score_calculator(stock_news_datasets)
sentiment_score_calculator(news_press_releases_datasets)

# Calculating aggregated sentiments(daily) for news, press release and social sentiments datasets for each company
aggregated_sentiments(stock_news_datasets)
aggregated_sentiments(news_press_releases_datasets)
aggregated_sentiments(social_sentiments_datasets)

# Reading in company data and Creating Joins

In [ ]:
# Setting company names 
company_names = ['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']

# Dictionary to store datasets
company_data = {}


# reading in files
for names in company_names:

    if names == 'GOOG':

        company_data[names] = {
            "stock_news": pd.read_csv(f"{names}_news_data.csv"),
            "press_releases": pd.read_csv(f"{names}_press_release_data.csv")
        }

    else:

        try:
            company_data[names] = {
                "stock_news": pd.read_csv(f"{names}_news_data.csv"),
                "press_releases": pd.read_csv(f"{names}_press_release_data.csv"),
                "social_sentiments": pd.read_csv(f"{names}_social_sentiment_data.csv")
            }

        except Exception as e:
            print(f"Error loading {names}: {e}")
            continue


dataset_variables = list(company_data.keys())

dataset_variables

['AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA']

In [ ]:
# Picking weighted_daily_sentiment and rename it to twitter_social_sentiment, press_release_sentiment and stock_news_sentiment for each dataset
for company in company_names:

    if company == 'GOOG':

        company_data[company]['stock_news'].rename(columns={'weighted_daily_sentiment' : 'stock_news_sentiment',
                                                        'published_date' : 'date'}, inplace=True)

        company_data[company]['press_releases'].rename(columns={'weighted_daily_sentiment' : 'press_release_sentiment',
                                                            'release_date' : 'date'}, inplace=True)
        
        # Selecting date & sentiment columns
        company_data[company]['stock_news'] = company_data[company]['stock_news'][['date', 'stock_news_sentiment']]
        company_data[company]['press_releases'] = company_data[company]['press_releases'][['date', 'press_release_sentiment']]

        # Dropping all duplicates
        company_data[company]['stock_news'].drop_duplicates(inplace=True)
        company_data[company]['press_releases'].drop_duplicates(inplace=True)


         # Displaying rows left for each dataset for the given company
        print('Duplicates on stock_news data removed. Number of rows remaining:', company_data[company]['stock_news'].shape[0])
        print('Duplicates on press_release data removed. Number of rows remaining:', company_data[company]['press_releases'].shape[0])

        # merging two datasets into one for Google as sentiment data is non-existent
        combined_news_data = company_data[company]['stock_news'].merge(company_data[company]['press_releases'], on='date', how='outer')

        # Writing processed data to news file for each company
        combined_news_data.to_csv(f"{company}_complete_news_data.csv", index=False)
        print(f"Finished updating {company} with weighted daily sentiment.")
    
    else:
        try:
            company_data[company]['stock_news'].rename(columns={'weighted_daily_sentiment' : 'stock_news_sentiment',
                                                            'published_date' : 'date'}, inplace=True)

            company_data[company]['press_releases'].rename(columns={'weighted_daily_sentiment' : 'press_release_sentiment',
                                                                'release_date' : 'date'}, inplace=True)

            company_data[company]['social_sentiments'].rename(columns={'weighted_daily_sentiment' : 'twitter_social_sentiment',
                                                                    'published_date' : 'date'}, inplace=True)


            # Selecting date & sentiment columns
            company_data[company]['stock_news'] = company_data[company]['stock_news'][['date', 'stock_news_sentiment']]
            company_data[company]['press_releases'] = company_data[company]['press_releases'][['date', 'press_release_sentiment']]
            company_data[company]['social_sentiments'] = company_data[company]['social_sentiments'][['date', 'twitter_social_sentiment']]

            # Dropping all duplicates
            company_data[company]['stock_news'].drop_duplicates(inplace=True)
            company_data[company]['press_releases'].drop_duplicates(inplace=True)
            company_data[company]['social_sentiments'].drop_duplicates(inplace=True)

            # Displaying rows left for each dataset for the given company
            print('Duplicates on stock_news data removed. Number of rows remaining:', company_data[company]['stock_news'].shape[0])
            print('Duplicates on press_release data removed. Number of rows remaining:', company_data[company]['press_releases'].shape[0])
            print('Duplicates on social_sentiments data removed. Number of rows remaining:', company_data[company]['social_sentiments'].shape[0])


            # merging three datasets into one for each company
            combined_news_data = company_data[company]['stock_news'].merge(company_data[company]['press_releases'], on='date', how='outer').merge(company_data[company]['social_sentiments'], on='date', how='outer')

            # Writing processed data to news file for each company
            combined_news_data.to_csv(f"{company}_complete_news_data.csv", index=False)
            print(f"Finished updating {company} with weighted daily sentiment.")

        except Exception as e:
            print(f"Error loading {company}: {e}")
            continue

Duplicates on stock_news data removed. Number of rows remaining: 2423
Duplicates on press_release data removed. Number of rows remaining: 570
Duplicates on social_sentiments data removed. Number of rows remaining: 93
Finished updating AAPL with weighted daily sentiment.
Duplicates on stock_news data removed. Number of rows remaining: 3290
Duplicates on press_release data removed. Number of rows remaining: 708
Duplicates on social_sentiments data removed. Number of rows remaining: 93
Finished updating AMZN with weighted daily sentiment.
Duplicates on stock_news data removed. Number of rows remaining: 2127
Duplicates on press_release data removed. Number of rows remaining: 167
Finished updating GOOG with weighted daily sentiment.
Duplicates on stock_news data removed. Number of rows remaining: 2039
Duplicates on press_release data removed. Number of rows remaining: 111
Duplicates on social_sentiments data removed. Number of rows remaining: 93
Finished updating META with weighted daily se

## PULLING COMMODITY DATA
##### Chosen commodities; Gold, Silver, Lithium, Copper, Palladium. 

In [ ]:
commodities = [
    {"commodity": "Gold", "symbol": "XAUUSD"},
    {"commodity": "Silver", "symbol": "XAGUSD"},
    {"commodity": "Lithium", "symbol": "LIT"},
    {"commodity": "Copper", "symbol": "HGUSD"},
    {"commodity": "Palladium", "symbol": "XPDUSD"}
]

In [ ]:
# Define current date
current_date = date.today()

# Iterate through commodities
for commodity in commodities:
    url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{commodity['symbol']}?apikey={fmp_api_key}"

    page = 0
    limit = 1000
    max_pages = 250   
    all_commodity_data = []

    params = {
                "symbol": commodity['symbol'],
               # "page": page,
                'from': '1980-01-01',  
                'to': current_date,
                'limit': limit
            }

    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {commodity['commodity']} on page {page}: {e}")
        break

    data = response.json()

    if not data:
        break

    all_commodity_data.extend(data.get("historical", []))


    # Create CSV file for commodity data
    csv_file = f"{commodity['commodity']}_commodity_data.csv"
    headers = ["date", "price", "volume", "open", "close"]
    formatted_data = []

    if all_commodity_data:
        for record in all_commodity_data:
            # Formats for date parsing
            formats = ['%Y-%m-%d %I:%M:%S %p', '%Y-%m-%d %H:%M:%S']

            if 'date' in record:
                for fmt in formats:
                    try:
                        record['date'] = datetime.strptime(record['date'], fmt).strftime('%d-%m-%Y')
                        break
                    except ValueError as e:
                        continue

            formatted_data.append({
                "date": record.get("date", ""),
                "price": record.get("close", ""),  
                "volume": record.get("volume", ""),
                "open": record.get("open", ""),
                "close": record.get("close", "")
            })

        with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            writer.writeheader()
            writer.writerows(formatted_data)

        print(f"Data for {commodity['commodity']} written to {csv_file} with {len(formatted_data)} records!")
    else:
        print(f"No data found for {commodity['commodity']}.")

print("All commodity data successfully written to their respective CSV files.")


### Pulling the Treasury Rates data

In [ ]:
# Define API key and URL
# The API key should be loaded before this, as you've mentioned it is already loaded elsewhere
base_url = "https://financialmodelingprep.com/api/v4/treasury"

# Defined date variables
end_date = "1980-01-01"  
current_date = datetime.today().strftime("%Y-%m-%d")

# Function to subtract months from a date
def subtract_months(start_date, months):
    new_date = datetime.strptime(start_date, "%Y-%m-%d") - timedelta(days=months*30)
    return new_date.strftime("%Y-%m-%d")

# Defined CSV file name
csv_file = "treasury_rates_data.csv"
headers = ["date", "month1", "month2", "month3", "month6", "year1", "year2", "year3", "year5", "year7", "year10", "year20", "year30"]
formatted_data = []

# Initialized the starting date (current date)
current_from_date = current_date

# Loop to fetch data in 3-month chunks
while current_from_date > end_date:
    # Calculated the "from" and "to" dates for the API call (3 months at a time)
    current_to_date = current_from_date
    current_from_date = subtract_months(current_from_date, 3)
    
    # Make sure the "from_date" does not go earlier than the end date
    if current_from_date < end_date:
        current_from_date = end_date

    # Define parameters for API request
    params = {
        "from": current_from_date,  
        "to": current_to_date,
        "apikey": fmp_api_key   
    }

    # Fetch treasury rates data
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Ensure we catch any HTTP errors
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching treasury rates for {current_from_date} to {current_to_date}: {e}")
        continue  

    # Process and format data
    if data:
        for record in data:
            formatted_data.append({
                "date": record.get("date", ""),
                "month1": record.get("month1", ""),
                "month2": record.get("month2", ""),
                "month3": record.get("month3", ""),
                "month6": record.get("month6", ""),
                "year1": record.get("year1", ""),
                "year2": record.get("year2", ""),
                "year3": record.get("year3", ""),
                "year5": record.get("year5", ""),
                "year7": record.get("year7", ""),
                "year10": record.get("year10", ""),
                "year20": record.get("year20", ""),
                "year30": record.get("year30", ""),
            })
        print(f"Fetched data for {current_from_date} to {current_to_date}")

# Write data to CSV after all batches
if formatted_data:
    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        writer.writeheader()
        writer.writerows(formatted_data)

    print(f"Treasury rates data written to {csv_file} with {len(formatted_data)} records!")
else:
    print("No treasury data found.")